# Гипотезы

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts

## Давайте еще раз посмотрим на наш датасет

In [2]:
df = pd.read_csv("C:\\Users\\user\\OneDrive\\Рабочий стол\\chistoe.csv")
df = df.drop('Unnamed: 0',axis=1)
df.columns = map(str.capitalize,df.columns)
df = df.fillna('No information')
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_14392\2168918879.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:\\Users\\user\\OneDrive\\Рабочий стол\\chistoe.csv")


,Event_id_cnty,Event_date,Year,Time_precision,Disorder_type,Event_type,Sub_event_type,Actor1,Assoc_actor_1,Inter1,...,Longitude,Geo_precision,Source,Source_scale,Notes,Fatalities,Tags,Timestamp,Index,Classification
0,ALG1,1997-01-01,1997,1,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,No information,2,...,2.789,1,Algeria Watch,Other,5 January: Beheading of 5 citizens in Douaouda...,5,No information,1582579226,0.624,Lower middle income
1,ALG2,1997-01-02,1997,1,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,No information,2,...,0.883,1,Algeria Watch,Other,Two citizens were beheaded in Hassasna.,2,No information,1582579226,0.624,Lower middle income
2,ALG3,1997-01-03,1997,1,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,No information,2,...,-0.290,1,Algeria Watch,Other,Two citizens were killed in a raid on the vill...,2,No information,1582579226,0.624,Lower middle income
3,ALG4,1997-01-04,1997,1,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,No information,2,...,2.829,1,Algeria Watch,Other,4 January: 16 citizens were murdered in the vi...,16,No information,1582579226,0.624,Lower middle income
4,ALG5,1997-01-05,1997,1,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,No information,2,...,2.789,1,Algeria Watch,Other,5 January: Killing of 18 citizens in the Olivi...,18,No information,1582579226,0.624,Lower middle income


### В первую очередь интерес представляют гипотезы, как-то связанные с целевыми переменными - Index и Classification. 

# <u>Гипотеза 1</u>

#### Для начала, давайте проверим гипотезу, сформулированную следующим образом:
> $$ H_0: \mu_H = \mu_L$$
> $$ H_1: \mu_H > \mu_L $$

#### Теперь немного о том, что мы будем проверять. Основная гипотеза(словами) сформулирована так: математическое ожидание индекса человеческого развития в регионах с более высоким доходом никак не отличается от математического ожидания индекса человеческого развития в регионах с более низким уровнем дохода. Соответственно, $\mu_H$ - матожидание индекса в более богатых регионах, $\mu_L$ - в более бедных.
- Важно заметить, что к первой категории мы будем относить регионы с метками __'High income'__ и __'Upper middle income'__. А ко второй - с __'Lower middle income'__ и __'Low income'__ (по переменной Classification).


#### Для начала, для более удобной работы, вытащим данные по двум категориям в два отдельных "датасета":

In [3]:
#для более богатых стран
high = df[(df['Classification']=='High income')|(df['Classification']=='Upper middle income')]
high = high[['Index','Classification']]
high.head()

,Index,Classification
1907,0.705,Upper middle income
1908,0.705,Upper middle income
1909,0.705,Upper middle income
1910,0.705,Upper middle income
1911,0.705,Upper middle income


In [4]:
#для более бедных стран
low = df[(df['Classification']=='Low income')|(df['Classification']=='Lower middle income')]
low = low[['Index','Classification']]
low.head()

,Index,Classification
0,0.624,Lower middle income
1,0.624,Lower middle income
2,0.624,Lower middle income
3,0.624,Lower middle income
4,0.624,Lower middle income


#### Давайте посмотрим на размерности получившихся датафреймов

In [5]:
high.shape,low.shape

((687404, 2), (1063312, 2))

#### Как мы видим, регионов с более низким уровнем дохода больше, чем с более высоким.

#### Теперь приступим к проверке гипотез. Будем проверять на уровне значимости в 5%.

In [6]:
mu_h = high['Index']
mu_l = low['Index']

#### Для проверки будем использовать тест Уэлча, он зашит в пакет scipy.stats. Откуда сразу найдём и p-value

In [7]:
a = sts.ttest_ind(mu_h,mu_l,equal_var=False,alternative='greater')
print(f'P-value:{a[1]}')

P-value:0.0


__Вывод__: Как мы видим, p-значение меньше, чем заданный уровень значимости, а следовательно,основная гипотеза о том, что математическое ожидание индекса человеческого развития в регионах с более высоким доходом никак не отличается от математического ожидания индекса человеческого развития в регионах с более низким уровнем дохода отвергается. 


# <u>Гипотеза 2</u>

#### Раз мы начали говорить о целевых переменных, давайте подробнее рассмотрим  'Index'.  Было бы интересно посмотреть, зависит ли эта переменная от переменной 'Fatalities', указывающая на количество смертей в конфликте.

#### Давайте проверим гипотезу, сформулированную следующим образом:
> $$ H_0: Corr (Fatalities,Index) = 0$$
> $$ H_1: Corr (Fatalities,Index) \neq 0$$

#### Где нулевая гипотеза свидетельствует о том, что зависимости между переменными нет, а альтернативная - что хоть какая-то зависимость есть

In [8]:
a = df['Fatalities']
b = df['Index']

#### Для проверки гипотезы будем использовать коэффициент корреляции Пирсона, аналогично, уровень значимости возьмём равный 5%

In [9]:
corr,p = sts.pearsonr(a,b)
print(f'P-value:{p}')

P-value:0.0


__Вывод__: Получили результат, что p-значение меньше нашего уровня значимости, следовательно, основная гипотеза отвергается. Но следует помнить, что корреляция Пирсона рассматривает только линейную связь между переменными, поэтому в нашем случае мы отвергаем только линейную связь между признаками.

# <u>Гипотеза 3</u>

#### Теперь вторая целевая переменная - Classification. Она категориальна, поэтому можем рассматривать её только в связи с другими категориальными. Например, рассмотрим с переменной 'Disorder_type'.

#### Давайте проверим гипотезу, сформулированную следующим образом:
> $$ H_0: Две \ переменные \ независимы$$
> $$ H_1: Существует \ зависимость \ между \ переменными$$

#### Для начала, составим таблицу сопряжённости:

In [10]:
pivot = pd.crosstab(index = df.Classification, columns = df.Disorder_type)
pivot.index = ['High income', 'Low income', 'Lower middle income','Upper middle income']
pivot.columns = ['Demonstrations', 'Political violence','Political violence; Demonstrations','Strategic developments']
pivot

,Demonstrations,Political violence,Political violence; Demonstrations,Strategic developments
High income,236951,10322,278,7583
Low income,45792,358914,1767,33022
Lower middle income,309769,278893,2433,32722
Upper middle income,212237,194624,714,24695


#### Для проверки данной гипотезы будем использовать Хи-квадрат критерий Пирсона, способ которого подшит в scipy.stats. Опять же, возьмём уровень значимости в 5%.

In [11]:
stat, p, dof, expected = sts.chi2_contingency(pivot)
print(f'P-value:{p}')

P-value:0.0


__Вывод__: Таким образом, основная гипотеза отвергается. А это значит, что всё-таки зависимость между переменными существует.

# <u>Гипотеза 4</u>

#### Интересно было бы посмотреть на то, какова вероятность того, что регион, в котором произошёл конфликт, имеет высокий или низкий доход. Давайте проверим следующую гипотезу:

> $$ H_0: p_H = p_L$$
> $$ H_1: p_H \neq p_L $$


#### Где $p_H$ - вероятность того, что в регионе высокий доход, а $p_L$ - низкий.

#### Для тестирования гипотезы мы будем использовать Z-статистику, так как данных у нас много, поэтому статистика хорошо аппроксимируется нормальным распределением. Посчитаем нужные для расчёта значения. Чтобы не добавлять ничего лишнего, возьмём те таблицы, что использовали при проверке первой гипотезы.

In [12]:
p_H = len(high)/len(df)
p_L = len(low)/len(df)
sigma_H = (p_H*(1-p_H))/len(high)
sigma_L = (p_L*(1-p_L))/len(low)

#### Значение нашей статистики:

In [13]:
z_obs = (p_H-p_L)/(np.sqrt(sigma_H+sigma_L))

#### Критическое значение на уровне значимости в 5%:

In [14]:
z_crit = sts.norm.ppf(0.95)
z_crit

1.6448536269514722

In [15]:
if abs(z_obs)>=z_crit:
    print('H0 отвергается')
else:
    print('H0 не отвергается')

H0 отвергается


__Вывод__: Значит, мы не можем утверждать, что регионов с высокими доходами столько же, сколько и регионов с низкими. И, действительно, давайте посмотрим на эти вероятности:

In [16]:
print(f'Вероятность того, что в регионе высокие доходы:{np.round(p_H,3)}')
print(f'Вероятность того, что в регионе низкие доходы:{np.round(p_L,3)}')

Вероятность того, что в регионе высокие доходы:0.393
Вероятность того, что в регионе низкие доходы:0.607


# <u>Гипотеза 5</u>

#### Ну и напоследок, рассмотрим следующую гипотезу:

> $$ H_0: \mu_1 = \mu_2$$
> $$ H_1: \mu_1 > \mu_2 $$

#### Где $\mu_1$  - математическое ожидание индекса для стран, где доминируют ненасильственные конфликты, a $\mu_2$ - наоборот. Тогда $H_0$ - отсутствие устойчивой разницы , а $H_1$ - страны с ненасильственными конфликтами в среднем имеют лучший показатель индекса человеческого развития

#### Анализировать будем по переменной Event_type, содержащей следующие значения:

In [17]:
df['Event_type'].unique()

array(['Violence against civilians', 'Battles',
       'Explosions/Remote violence', 'Protests', 'Strategic developments',
       'Riots'], dtype=object)

Напомним, что
> - Violence against civilians - Насилие в отношении гражданских лиц
> - Battles - Драки
> - Explosions/Remote violence - Взрывы/Насилие на расстоянии
> - Protests - Протесты
> - Strategic developments - Стратегические разработки
> - Riots - Массовые беспорядки

#### Отнесём Protests и Strategic developments к ненасильственным конфликтам, а остальное - к насильственным.

In [18]:
nas = df[(df['Event_type']=='Violence against civilians')|(df['Event_type']=='Battles')|(df['Event_type']=='Explosions/Remote violence')|(df['Event_type']=='Riots')]
ne_nas = df[(df['Event_type']=='Protests')|(df['Event_type']=='Strategic developments')]

In [19]:
nas = nas['Index']
ne_nas = ne_nas['Index']

#### Будем использовать Z-статистику, так как данных достаточно много. Уровень значимости - 5%.

In [20]:
mu_1 = ne_nas.mean()
mu_2 = nas.mean()

In [21]:
sigma_1 = (ne_nas.std())**2/len(ne_nas)
sigma_2 = (nas.std())**2/len(nas)

In [22]:
Z_obs = (mu_1-mu_2)/(np.sqrt(sigma_1+sigma_2))

In [23]:
Z_crit = sts.norm.ppf(0.95)

In [24]:
if Z_obs>Z_crit:
    print('H0 отвергается')
else:
    print('H0 не отвергается')

H0 отвергается


__Вывод__: Таким образом, основная гипотеза о том, что нет разницы между математическим ожиданием индекса человеческого развития для стран, где преобладают ненасильственные конфликты, и для тех, где преобладают насильственные, отвергается